Kalman Filter, Gerrik
Load the data

In [ ]:
import torch
import numpy as NP
import pandas as PD
import os, scipy, pickle
import matplotlib.pyplot as PLT
import Neural_Decoding as ND
from Neural_Decoding.preprocessing_funcs import bin_spikes
from Neural_Decoding.preprocessing_funcs import bin_output
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho
from Neural_Decoding.decoders import KalmanFilterDecoder

T_B=.05 #Size of time bins in seconds

for File_Add in ["C:/Users/gerri/Coding_Folder/Data_Folder/DS_Final_Data/example_data_hc.pickle"]:
    Mat_Dict=scipy.io.loadmat(File_Add,spmatrix=False)
    Keys=list(Mat_Dict.keys())
    """
    spike_times:    When neurons Fired.
    vels       :    X and Y velocities
    vel_times  :    When vel times were taken.
    pos        :    X and Y positions of a cursor
    acc        :    X and Y accuracy of cursor
    """
    
    #Reformat matlab variables to pandas DF.
    DF=dict()
    DF["Vel_x"]=Mat_Dict["vels"][:,0]
    DF["Vel_y"]=Mat_Dict["vels"][:,1]
    DF["Vel_t"]=NP.concat(Mat_Dict["vel_times"])
    DF["Pos_x"]=Mat_Dict["pos"][:,0]
    DF["Pos_y"]=Mat_Dict["pos"][:,1]
    DF["acc_x"]=Mat_Dict["acc"][:,0]
    DF["acc_y"]=Mat_Dict["acc"][:,1]
    Spike_Times=NP.sort(NP.reshape(Mat_Dict["spike_times"],shape=-1))
    PD.DataFrame(DF)

    neural_data=bin_spikes(
        spike_times=DF["Vel_t"],
        dt=T_B,
        wdw_start=DF["Vel_t"][0],
        wds_end=DF["Vel_t"][-1])
    
    vels_binned=bin_output(
        outputs=Mat_Dict["vels"],
        output_times=DF["Vel_t"],
        dt=         T_B,            #Time bin size==0.05 seconds
        wdw_start=  DF["Vel_t"][0], #Start at first sample
        wdw_end=    DF["Vel_t"][-1],#End at last sample
        downsample_factor=1)

ModuleNotFoundError: No module named 'Neural_Decoding'

As can be seen above, both files contain the same variables.

In [ ]:
import torch
import numpy as NP
import pandas as PD
import os, scipy, pickle
import matplotlib.pyplot as PLT
import Neural_Decoding as ND
from Neural_Decoding.preprocessing_funcs import bin_spikes
from Neural_Decoding.preprocessing_funcs import bin_output
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho
from Neural_Decoding.decoders import KalmanFilterDecoder

with open("C:/Users/gerri/Coding_Folder/Data_Folder/DS_Final_Data/example_data_hc.pickle") as File:
    neural_data,pos_binned=pickle.load(File,encoding='latin1')
    print(f"neural_data \n{NP.shape(neural_data)}"+"\n"+f"pos_binned \n{NP.shape(pos_binned)}")
    
    """
    neural_data:    When neurons Fired. num_bins x num_neurons. 1 when fired, 0 when not.
    pos_binned       :    X and Y velocities
    """
    lag=0
    nd_sum=NP.nansum(neural_data,axis=0) #Total number of spikes of each neuron
    nd_mean=NP.mean(nd_sum)
    nd_std=NP.std(nd_sum)
    nd_thresh=nd_mean-nd_std*2
    rmv_nrn=NP.where(nd_sum<100) #Find neurons who have less than 2 std of dataset. Gerrik
    rmv_nrn_std=NP.where(nd_sum<nd_thresh) #Find neurons who have less than 100 spikes total
    print(f"rmv_nrn \n{NP.shape(rmv_nrn)}"+"\n"+f"rmv_nrn_std \n{NP.shape(rmv_nrn_std)}")

    Covariate=NP.delete(neural_data,rmv_nrn,1)
    Covariate_G=NP.delete(neural_data,rmv_nrn,1)

    temp=NP.diff(pos_binned,axis=0)
    vels_binned=NP.concatenate((temp,temp[-1:,:]),axis=0)

    #We now determine acceleration
    temp2=NP.diff(vels_binned,axis=0)
    acc_binned=NP.concatenate((temp2,temp2[-1:,:]),axis=0)

    y_kf=NP.concatenate((pos_binned,vels_binned,acc_binned),axis=1)
